In [1]:
import argparse
import math
import sys
import time
import copy

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
K.set_image_dim_ordering('th')
print(K.image_data_format())

## required for efficient GPU use
import tensorflow as tf
from keras.backend import tensorflow_backend
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session = tf.Session(config=config)
tensorflow_backend.set_session(session)
## required for efficient GPU use

import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

c:\users\anusha\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


channels_first


In [2]:
model_path='D:/AI/Jinoh Kim/one_class_svm/intrusion/Final_data/cnn_deep_mon.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

In [3]:
# get data

class dataset:
    monday = pd.read_pickle("D:/AI/Jinoh Kim/one_class_svm/intrusion/Final_data/preprocessed_monday_reduced_final.pkl")
    wednesday = pd.read_pickle("D:/AI/Jinoh Kim/one_class_svm/intrusion/Final_data/preprocessed_wednesday_final_data.pkl")
    
    
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['label_0','label_1']
    
    x_input = dataset.monday.drop(output_columns_2labels, axis = 1)
    y_output = dataset.monday.loc[:,output_columns_2labels]

    x_test_input = dataset.wednesday.drop(output_columns_2labels, axis = 1)
    y_test = dataset.wednesday.loc[:,output_columns_2labels]
    
    ss = pp.StandardScaler()
    
    x_train = ss.fit_transform(x_input)
    x_test = ss.fit_transform(x_test_input)
    
    y_train = y_output.values
    y_test = y_test.values
        
    print('x_train shape: {}'.format(x_train.shape))
    print('y_train shape: {}'.format(y_train.shape))
    print('x_test shape: {}'.format(x_test.shape))
    print('y_test shape: {}'.format(y_test.shape))

x_train shape: (52991, 78)
y_train shape: (52991, 2)
x_test shape: (69270, 78)
y_test shape: (69270, 2)


In [4]:
#reshape data
X_train = np.reshape(preprocess.x_train, (preprocess.x_train.shape[0], preprocess.x_train.shape[1], 1))
X_test = np.reshape(preprocess.x_test, (preprocess.x_test.shape[0], preprocess.x_test.shape[1], 1))

print('train shape after reshape: {}'.format(X_train.shape))
print('test shape after reshape: {}'.format(X_test.shape))

train shape after reshape: (52991, 78, 1)
test shape after reshape: (69270, 78, 1)


In [5]:
#size of parameters
batch_size = 1000
num_classes = 2
epochs = 10
filter_size=3
#noise = 1
droprate=0.50

In [6]:
#Start Neural Network
model = Sequential()

#convolution 1st layer
model.add(Conv1D(64, kernel_size=(filter_size), padding="same",
                 activation='relu',
                 input_shape=(78, 1)))
model.add(BatchNormalization())
model.add(Dropout(droprate))
          
#convolution 2nd layer
model.add(Conv1D(128, kernel_size=(filter_size), activation='relu', border_mode="same"))
model.add(BatchNormalization())
model.add(MaxPooling1D(strides=1))
model.add(Dropout(droprate))

#convolution 3rd layer
model.add(Conv1D(256, kernel_size=(filter_size), activation='relu', border_mode="same"))
model.add(BatchNormalization())
model.add(MaxPooling1D(strides=1))
model.add(Dropout(droprate))

#FCN 1st layer
model.add(Flatten())
model.add(Dense(64,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN 2nd layer
model.add(Dense(32,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN 3rd layer
model.add(Dense(16,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN final layer
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])

model.summary()

c:\users\anusha\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, kernel_size=3, activation="relu", padding="same")`
  if sys.path[0] == '':
c:\users\anusha\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, kernel_size=3, activation="relu", padding="same")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 78, 64)            256       
_________________________________________________________________
batch_normalization_1 (Batch (None, 78, 64)            256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 78, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 78, 128)           24704     
_________________________________________________________________
batch_normalization_2 (Batch (None, 78, 128)           512       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 77, 128)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 77, 128)           0         
__________

In [7]:
#Save Model=ON
history = model.fit(X_train, preprocess.y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, preprocess.y_test),shuffle=True,callbacks=callbacks)

score = model.evaluate(X_test, preprocess.y_test, verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 52991 samples, validate on 69270 samples
Epoch 1/10
52991/52991 [==============================] - 1371s 26ms/step - loss: 0.8921 - acc: 0.4532 - val_loss: 0.6157 - val_acc: 0.8997
Epoch 2/10
52991/52991 [==============================] - 1248s 24ms/step - loss: 0.5395 - acc: 0.8463 - val_loss: 0.4490 - val_acc: 0.9162
Epoch 3/10
52991/52991 [==============================] - 992s 19ms/step - loss: 0.3348 - acc: 0.9400 - val_loss: 0.3690 - val_acc: 0.9164
Epoch 4/10
52991/52991 [==============================] - 989s 19ms/step - loss: 0.2218 - acc: 0.9788 - val_loss: 0.3122 - val_acc: 0.9164
Epoch 5/10
52991/52991 [==============================] - 670s 13ms/step - loss: 0.1571 - acc: 0.9935 - val_loss: 0.2898 - val_acc: 0.9164
Epoch 6/10
52991/52991 [==============================] - 506s 10ms/step - loss: 0.1187 - acc: 0.9965 - val_loss: 0.2883 - val_acc: 0.9164
Epoch 7/10
52991/52991 [==============================] - 498s 9ms/step - loss: 0.0936 - acc: 0.9986 - val_loss: 0

In [8]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

y_pred = model.predict_classes(X_test)
print(y_pred)

p=model.predict_proba(X_test)

target_names = ['0', '1']
print(classification_report(np.argmax(preprocess.y_test, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(preprocess.y_test, axis=1), y_pred))

[[0.93999684 0.06000319]
 [0.96740615 0.03259385]
 [0.97215074 0.02784929]
 ...
 [0.9710032  0.02899683]
 [0.97426087 0.02573917]
 [0.9692654  0.03073466]]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
             precision    recall  f1-score   support

          0     0.9164    1.0000    0.9564     63478
          1     0.0000    0.0000    0.0000      5792

avg / total     0.8398    0.9164    0.8764     69270

[[63478     0]
 [ 5792     0]]


c:\users\anusha\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
